# Analyzing CIA Factbook Data Using SQL

In this project, we'll work with data from the [CIA World Factbook](https://www.cia.gov/library/publications/the-world-factbook/), a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information like:

- population - The population as of 2015.
- population_growth - The annual population growth rate, as a percentage.
- area - The total land and water area.

The SQLite database, factbook.db, is available in the repository.

We use the following code to connect our Jupyter Notebook to our database file:

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

Note that we must use %%sql in every SQL cell, or else the code will be read as iPython.

## Overview of the Data

In [2]:
%%sql
SELECT *
FROM sqlite_master 
WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


## Summarizing the Data

In [3]:
%%sql 
SELECT *
FROM facts 
LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46



Here are the descriptions for some of the columns:

- name - The name of the country.
- area - The total land and sea area of the country.
- population - The country's population.
- population_growth- The country's population growth as a percentage.
- birth_rate - The country's birth rate, or the number of births a year per 1,000 people.
- death_rate - The country's death rate, or the number of death a year per 1,000 people.
- area- The country's total area (both land and water).
- area_land - The country's land area in square kilometers.
- area_water - The country's waterarea in square kilometers.

Let's start by calculating some summary statistics and look for any outlier countries.

## Identifying Outliers

In [4]:
%%sql
SELECT MIN(population) 'min_pop', MAX(population) 'max_pop', MIN(population_growth) 'min_pop_growth', MAX(population_growth) 'max_pop_growth'
FROM facts;

 * sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


A few things stick out from the summary statistics:

- there's a country with a population of 0
- there's a country with a population of 7256490011 (or more than 7.2 billion people)

So we find the countries with 0 population and maximum population.

In [5]:
%%sql
SELECT name
FROM facts 
WHERE population = (SELECT MIN(population)
                   FROM facts);

 * sqlite:///factbook.db
Done.


name
Antarctica


In [6]:
%%sql
SELECT name
FROM facts 
WHERE population = (SELECT MAX(population)
                   FROM facts);

 * sqlite:///factbook.db
Done.


name
World


It seems like the table contains a row for the whole world, which explains the population of over 7.2 billion. It also seems like the table contains a row for Antarctica, which explains the population of 0.

We move on to calculating Averages now.

In [7]:
%%sql
SELECT ROUND(AVG(population), 2) 'avg_pop', ROUND(AVG(area), 2) 'avg_area'
FROM facts;

 * sqlite:///factbook.db
Done.


avg_pop,avg_area
62094928.32,555093.55


We will now identify the countries that have:
- above average values for population
- below average values for area


### Countries with population more than average population

In [8]:
%%sql
SELECT name
FROM facts
WHERE population > (SELECT AVG(population)
                   FROM facts);

 * sqlite:///factbook.db
Done.


name
Bangladesh
Brazil
China
"Congo, Democratic Republic of the"
Egypt
Ethiopia
France
Germany
India
Indonesia


### Countries with area less than average area

In [9]:
%%sql
SELECT name
FROM facts
WHERE area < (SELECT AVG(area)
                   FROM facts);

 * sqlite:///factbook.db
Done.


name
Albania
Andorra
Antigua and Barbuda
Armenia
Austria
Azerbaijan
"Bahamas, The"
Bahrain
Bangladesh
Barbados


## Exploring water to land ratios

In [10]:
%%sql
SELECT name, CAST(area_water as float)/area_land 'ratio_w:l'
FROM facts;

 * sqlite:///factbook.db
Done.


name,ratio_w:l
Afghanistan,0.0
Albania,0.04927366961092051
Algeria,0.0
Andorra,0.0
Angola,0.0
Antigua and Barbuda,0.0
Argentina,0.015971849204696183
Armenia,0.054604120129064286
Australia,0.007669578121135597
Austria,0.017296379404451452


### Countries with more water than land

In [11]:
%%sql
SELECT name
FROM facts
WHERE area_water > area_land;

 * sqlite:///factbook.db
Done.


name
British Indian Ocean Territory
Virgin Islands


### Countries with the most population growth

In [12]:
%%sql
SELECT name, population_growth
FROM facts
ORDER BY population_growth DESC
LIMIT 5;

 * sqlite:///factbook.db
Done.


name,population_growth
South Sudan,4.02
Malawi,3.32
Burundi,3.28
Niger,3.25
Uganda,3.24


### Countries with more death rate than birth rate

In [13]:
%%sql
SELECT name
FROM facts
WHERE death_rate > birth_rate;

 * sqlite:///factbook.db
Done.


name
Austria
Belarus
Bosnia and Herzegovina
Bulgaria
Croatia
Czech Republic
Estonia
Germany
Greece
Hungary
